In [ ]:
#1. Đọc và lọc dữ liệu

In [4]:
import pandas as pd
import openpyxl
from sklearn.preprocessing import StandardScaler

file_path = "Book2.xlsx"   
df = pd.read_excel(file_path,sheet_name='khaosatNUOC')

# print(df.loc[0:10])

df_small = df[["Column4","Column5"]]
df_small["Column5"] = (
    df_small["Column5"]
    .str.replace(",", ".", regex=False)        # đổi dấu phẩy thành dấu chấm
    .str.extract(r"(\d+\.?\d*)")               # lấy phần số
    .astype(float)                             # ép về float
)

mean_value   = df_small["Column5"].mean()    # trung bình
median_value = df_small["Column5"].median()  # trung vị
var_value    = df_small["Column5"].var()     # phương sai 
std_value    = df_small["Column5"].std()     # độ lệch chuẩn 

print("Trung bình:", mean_value)
print("Trung vị:", median_value)
print("Phương sai:", var_value)
print("Độ lệch chuẩn:", std_value)


Trung bình: 1.7524752475247525
Trung vị: 1.5
Phương sai: 0.18311881188118814
Độ lệch chuẩn: 0.42792383887928953


C:\Users\CONG\AppData\Local\Temp\ipykernel_13648\2002369003.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small["Column5"] = (


In [ ]:
#2. Chuẩn hoá z-core

In [5]:
import pandas as pd
import numpy as np
from scipy import stats
# 1.Hàm tính z-score cho một cột
df = pd.read_excel(file_path,sheet_name="khaosatNUOC")
def zscore_column(series, ddof=1):
    mu = series.mean()
    sd = series.std(ddof=ddof)
    if sd == 0 or np.isnan(sd):
        return pd.Series(np.zeros(len(series)), index=series.index)
    return (series - mu) / sd

# Tính Z-score cho cột số lượng nước uống (Column5)
z_df = pd.DataFrame({'Column5_zscore': zscore_column(df_small['Column5'].dropna(), ddof=1)})

# In toàn bộ Z-score
print(z_df)

     Column5_zscore
0         -0.590000
1          0.578432
2          0.578432
3         -0.590000
4         -0.590000
..              ...
96         0.578432
97        -0.590000
98         0.578432
99         0.578432
100        1.746864

[101 rows x 1 columns]


In [ ]:
# 3. Ước lượng khoảng tin cậy 

In [6]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import StandardScaler

lst = list(df_small["Column5"])[1:]

n = len(lst)
trung_binh_mau = np.mean(lst)
phuong_sai_mau = np.var(lst, ddof=1)
do_lech_chuan_mau = np.sqrt(phuong_sai_mau)
sai_so_chuan = do_lech_chuan_mau / np.sqrt(n)

muc_tin_cay = 0.95
alpha = 1 - muc_tin_cay
z_toi_han = stats.norm.ppf(1 - alpha/2)
can_duoi = trung_binh_mau - z_toi_han * sai_so_chuan
can_tren = trung_binh_mau + z_toi_han * sai_so_chuan

print(f"Số lượng mẫu (n): {n}")
print(f"Giá trị trung bình mẫu: {trung_binh_mau:.4f} lít")
print(f"Độ lệch chuẩn mẫu hiệu chỉnh (s): {do_lech_chuan_mau:.4f} lít")
print(f"Giá trị Z_critical: {z_toi_han:.4f}")
print(f"Khoảng tin cậy 95% cho trung bình dân số: ({can_duoi:.4f}, {can_tren:.4f}) lít")


Số lượng mẫu (n): 100
Giá trị trung bình mẫu: 1.7550 lít
Độ lệch chuẩn mẫu hiệu chỉnh (s): 0.4293 lít
Giá trị Z_critical: 1.9600
Khoảng tin cậy 95% cho trung bình dân số: (1.6709, 1.8391) lít


In [7]:
# 4. Kiểm định giả thuyết với mức ý nghĩa 5%

In [ ]:
import numpy as np
from scipy import stats

H_0 = 1.8
alpha = 0.05
lst = list(df_small["Column5"])[1:] # type: ignore
n = len(lst)
trung_binh_mau = np.mean(lst)
phuong_sai_mau = np.var(lst, ddof=1)
do_lech_chuan_mau = np.sqrt(phuong_sai_mau)
sai_so_chuan = do_lech_chuan_mau / np.sqrt(n)
z_qs = (trung_binh_mau - H_0) / (do_lech_chuan_mau / np.sqrt(n))

print(f"Giá trị trung bình mẫu (x_bar): {trung_binh_mau:.4f}")
print(f"Độ lệch chuẩn mẫu hiệu chỉnh (s): {do_lech_chuan_mau:.4f}")
print(f"Giá trị Z quan sát (z_qs): {z_qs:.4f}")

# Tìm giá trị Z tới hạn (z_critical)
z_critical = stats.norm.ppf(1 - alpha/2)
print(f"Giá trị Z tới hạn (z_critical): {z_critical:.4f}")
# Kiểm tra điều kiện bác bỏ H0
if abs(z_qs) > z_critical:
    print("Chúng ta bác bỏ giả thuyết H0. Có bằng chứng cho thấy lượng nước trung bình khác biệt đáng kể so với 1.8 lít.")
else:
    print("Chúng ta chưa có đủ bằng chứng để bác bỏ giả thuyết H0. Lượng nước uống trung bình của mẫu không khác biệt đáng kể so với 1.8 lít.")

Giá trị trung bình mẫu (x_bar): 1.7550
Độ lệch chuẩn mẫu hiệu chỉnh (s): 0.4293
Giá trị Z quan sát (z_qs): -1.0482
Giá trị Z tới hạn (z_critical): 1.9600
Chúng ta chưa có đủ bằng chứng để bác bỏ giả thuyết H0. Lượng nước uống trung bình của mẫu không khác biệt đáng kể so với 1.8 lít.
